In [2]:
# ---------------------------------------------------------------
#  DEF80 calculations for σ-LEM hierarchy
#  – first-order curves include Z ≥ –1/σ truncation factor –
# ---------------------------------------------------------------
import numpy as np
from scipy.optimize import fsolve
import pandas as pd
from math import erfc
v_erfc = np.vectorize(erfc)  # vectorised erfc

# ---------- Fixed parameters ------------------------------------
alpha_ctrl = 0.0252           # Gy⁻¹
beta_ctrl  = 0.00130          # Gy⁻²
Kc         = 2.11             # mM⁻¹  (100 kVp beam)

# ---------- Experimental DEF80 for comparison -------------------
experimental_def80 = {0.25: 1.17, 0.50: 1.47, 1.00: 1.94}

# ---------- Helper functions ------------------------------------
def sigma(c):                                # log-normal width
    return np.sqrt(2 * np.log1p(Kc * c))


def _trunc_factor(d, s):
    """
    𝔽(D; σ) = erfc( … ) / erfc(–1/(σ√2))
    factor that removes the Z < –1/σ slice after first-order expansion.
    """
    A = np.sqrt(1 + 2 * beta_ctrl * s**2 * d**2)
    shift = (-1 / s) + (s * d * (alpha_ctrl + 2 * beta_ctrl * d)) / (1 + 2 * beta_ctrl * s**2 * d**2)
    return v_erfc(A / np.sqrt(2) * shift) / erfc(-1 / (s * np.sqrt(2)))


# ---------- σ-LEM survival models --------------------------------
def S_baseline(d):
    return np.exp(-alpha_ctrl * d - beta_ctrl * d**2)


def S_variance_only(d, c):
    """First-order, variance-only *with truncation*"""
    s   = sigma(c)
    Den = 1 + 2 * beta_ctrl * s**2 * d**2
    Num = -alpha_ctrl * d - beta_ctrl * d**2 + (alpha_ctrl**2 * s**2 * d**2) / (2 * Den)
    return np.exp(Num) / np.sqrt(Den) * _trunc_factor(d, s)


def S_mixed_term(d, c):
    """First-order, mixed term α→α⋆  *with truncation*"""
    s   = sigma(c)
    a_  = alpha_ctrl + 2 * beta_ctrl * d
    Den = 1 + 2 * beta_ctrl * s**2 * d**2
    Num = -alpha_ctrl * d - beta_ctrl * d**2 + (a_ * s * d)**2 / (2 * Den)
    return np.exp(Num) / np.sqrt(Den) * _trunc_factor(d, s)


def S_complete_second_order(d, c):
    """Complete 2nd-order (μ = 0)"""
    s2 = sigma(c)**2
    Den = 1 + s2 * d * (alpha_ctrl + 4 * beta_ctrl * d)
    Num = -alpha_ctrl * d - beta_ctrl * d**2 + s2 * d**2 * (alpha_ctrl + 2 * beta_ctrl * d)**2 / (2 * Den)
    return np.exp(Num) / np.sqrt(Den)


# ---------- Root-finder for D80 ----------------------------------
def _find_d80(func, *args):
    """Return dose where S = 0.8; uses fsolve with bisection fallback."""
    def eq(d):
        return func(d, *args) - 0.8 if args else func(d) - 0.8
    try:
        d80 = float(fsolve(eq, 3.0)[0])
        if abs(eq(d80)) < 1e-3:
            return d80
    except Exception:
        pass  # fall through to bisection
    lo, hi = 0.1, 15.0
    for _ in range(100):
        mid = 0.5 * (lo + hi)
        val = eq(mid)
        if abs(val) < 1e-4:
            return mid
        if val > 0:
            lo = mid
        else:
            hi = mid
    return mid


# ---------- DEF80 & LaTeX generation -----------------------------
def calculate_def80_table():
    concentrations = [0.25, 0.50, 1.00]
    models = {'Variance-only': S_variance_only,
              'Mixed term'  : S_mixed_term,
              'Complete 2nd': S_complete_second_order}

    d80_baseline = _find_d80(S_baseline)
    print(f"Baseline D80 (0 mM): {d80_baseline:.4f} Gy\n")

    rows = []
    for conc in concentrations:
        row = {'Concentration (mM)': conc,
               'Experimental'       : experimental_def80[conc]}
        for name, func in models.items():
            d80 = _find_d80(func, conc)
            DEF = d80_baseline / d80
            err = 100 * (DEF - experimental_def80[conc]) / experimental_def80[conc]
            row[f'{name}_D80']    = d80
            row[f'{name}_DEF80']  = DEF
            row[f'{name}_Error']  = err
            print(f"{conc:.2f} mM – {name}: D80 = {d80:.4f} Gy   DEF80 = {DEF:.3f}   "
                  f"(exp {experimental_def80[conc]:.3f}, error {err:+.1f}%)")
        print()
        rows.append(row)
    return pd.DataFrame(rows), d80_baseline


def _fmt_val(val, err):
    if np.isnan(val) or np.isnan(err):
        return "---"
    color = r"\textcolor{darkgreen}" if abs(err) < 10 else (
            r"\textcolor{orange}"    if abs(err) < 25 else r"\textcolor{red}")
    return f"{color}{{{val:.3f}}}"


def generate_latex(df, d80_baseline):
    """Return LaTeX tables as one string."""
    tab1 = [r"\begin{table}[h!]",
            r"\centering",
            rf"\caption{{DEF$_{{80}}$ comparison; baseline D$_{{80}}$ = {d80_baseline:.3f}\,Gy.}}",
            r"\label{tab:def80}",
            r"\begin{tabular}{@{}c|c|ccc@{}}\toprule",
            r"\multirow{2}{*}{Conc. (mM)} & \multirow{2}{*}{Exp.} & \multicolumn{3}{c}{σ-LEM models}\\",
            r"\cmidrule(l){3-5}",
            r"& & Variance & Mixed & Complete 2nd\\\midrule"]
    tab2 = [r"\begin{table}[h!]",
            r"\centering",
            r"\caption{Percentage errors w.r.t. experimental DEF$_{80}$.}",
            r"\label{tab:def80err}",
            r"\begin{tabular}{@{}c|ccc@{}}\toprule",
            r"Conc. (mM) & Variance & Mixed & Complete 2nd\\\midrule"]
    tab3 = [r"\begin{table}[h!]",
            r"\centering",
            r"\caption{Best model (minimum |error|) per concentration.}",
            r"\label{tab:bestmodel}",
            r"\begin{tabular}{@{}c|c|c@{}}\toprule",
            r"Conc. (mM) & Best model & Error (\%)\\\midrule"]

    for _, r in df.iterrows():
        c = r['Concentration (mM)']
        expv = r['Experimental']
        vdef, verr = r['Variance-only_DEF80'],  r['Variance-only_Error']
        mdef, merr = r['Mixed term_DEF80'],     r['Mixed term_Error']
        cdef, cerr = r['Complete 2nd_DEF80'],   r['Complete 2nd_Error']
        tab1.append(f"{c:.2f} & {expv:.3f} & {_fmt_val(vdef,verr)} & {_fmt_val(mdef,merr)} & {_fmt_val(cdef,cerr)}\\\\")
        tab2.append(f"{c:.2f} & {verr:+.1f} & {merr:+.1f} & {cerr:+.1f}\\\\")
        best = min({'Variance-only': abs(verr), 'Mixed term': abs(merr), 'Complete 2nd': abs(cerr)}.items(),
                   key=lambda x: x[1])
        tab3.append(f"{c:.2f} & {best[0]} & {np.sign(best[1])*best[1]:+.1f}\\\\")
    for tab in (tab1, tab2, tab3):
        tab.extend([r"\bottomrule", r"\end{tabular}", r"\end{table}", ""])
    return "\n".join("\n".join(t) for t in (tab1, tab2, tab3))


# ---------- Main --------------------------------------------------
def main():
    print("="*66, "\nDEF80 CALCULATION – σ-LEM hierarchy\n", "="*66, sep='')
    df, d80_base = calculate_def80_table()
    latex = generate_latex(df, d80_base)

    print("="*66, "\nLATEX TABLES\n", "="*66, sep='')
    print(latex)

    df.to_csv("DEF80_results.csv", index=False)
    with open("DEF80_tables.tex", "w") as f:
        f.write(latex)
    print("\nFiles written:  DEF80_results.csv   DEF80_tables.tex")

if __name__ == "__main__":
    main()


DEF80 CALCULATION – σ-LEM hierarchy
Baseline D80 (0 mM): 6.6046 Gy

0.25 mM – Variance-only: D80 = 5.0754 Gy   DEF80 = 1.301   (exp 1.170, error +11.2%)
0.25 mM – Mixed term: D80 = 5.2436 Gy   DEF80 = 1.260   (exp 1.170, error +7.7%)
0.25 mM – Complete 2nd: D80 = 4.6437 Gy   DEF80 = 1.422   (exp 1.170, error +21.6%)

0.50 mM – Variance-only: D80 = 4.3912 Gy   DEF80 = 1.504   (exp 1.470, error +2.3%)
0.50 mM – Mixed term: D80 = 4.5459 Gy   DEF80 = 1.453   (exp 1.470, error -1.2%)
0.50 mM – Complete 2nd: D80 = 3.9332 Gy   DEF80 = 1.679   (exp 1.470, error +14.2%)

1.00 mM – Variance-only: D80 = 3.7996 Gy   DEF80 = 1.738   (exp 1.940, error -10.4%)
1.00 mM – Mixed term: D80 = 3.9316 Gy   DEF80 = 1.680   (exp 1.940, error -13.4%)
1.00 mM – Complete 2nd: D80 = 3.2842 Gy   DEF80 = 2.011   (exp 1.940, error +3.7%)

LATEX TABLES
\begin{table}[h!]
\centering
\caption{DEF$_{80}$ comparison; baseline D$_{80}$ = 6.605\,Gy.}
\label{tab:def80}
\begin{tabular}{@{}c|c|ccc@{}}\toprule
\multirow{2}{*}{C